In [1]:
import sys
sys.path.append("..")

In [2]:
import os
import tensorflow as tf
import numpy as np
from preprocess import *

# Simulate Test Sentence

In [3]:
#########################
# 5 Simulated sentences #
#########################

samp_freq = 10
max_len = 200
input_vocab_size = 500
d_lower, d_upper = 0., 100. 
d_mu, d_sigma = 30., 10.
min_frame_len = 10
asn_upper = np.log(.95 * d_upper)
asn_lower = np.log(min_frame_len/1000.)
num_buckets = 50
inc = (asn_upper - asn_lower)/(num_buckets-2)
sen_n = 1
batch_size = 1

duration_test_sen = [create_duration_sentence(max_len,input_vocab_size,d_lower, d_upper, d_mu, d_sigma, 
                            asn_upper, asn_lower, num_buckets, batch_size) for i in range(sen_n/batch_size)]

max_len = 512
voiced_thresh = 1000.
lower,upper = -5000.,5000.
mu,sigma = 0., 500.
sen_n = 5

frequency_test_sen = [create_frequency_sentence(max_len,input_vocab_size,mu,sigma,lower,upper,
                                            input_vocab_size,voiced_thresh,batch_size) for i in range(sen_n/batch_size)]
#print test_sentences[0]['durations']

preprocess.py:63: RuntimeWarning: divide by zero encountered in log
  log_duration = np.log(duration)


# Predict 

In [4]:
duration_model_path = "../weights/train_duration_model_notebook/model.ckpt"
frequency_model_path = "../weights/train_frequency_model_notebook/model.ckpt"


def duration_predict(model_path,sentences):
    tf.reset_default_graph()
    saver = tf.train.import_meta_graph(model_path+".meta")
    graph = tf.get_default_graph()

    with tf.Session() as sess:
        saver.restore(sess,model_path)
        trans_params = tf.get_collection("trans_params")[0] 
        for s in sentences:
            pred_durations = sess.run("ReverseSequence_1:0", feed_dict={            
                                        "p_phonemes:0": s['phonemes'], 
                                        "p_seq_len:0": s['phonemes_seq_len'], 
                                        "p_spk_ids:0": np.ones((batch_size)),
                                        "p_params:0":trans_params.eval()
                                        })
            s['pred_durations'] = pred_durations
            s['phoneme_frames'] = phonemes_to_frames(s['phonemes'],pred_durations,samp_freq,asn_lower,inc)
        return sentences

    
def frequency_predict(model_path,sentences):
    tf.reset_default_graph()
    saver = tf.train.import_meta_graph(model_path+".meta")
    graph = tf.get_default_graph()

    with tf.Session() as sess:
        saver.restore(sess,model_path)
        for s in sentences:
            pred_voiced, pred_f_zero = sess.run(["voiced_probability_model_1/dense/Sigmoid:0",
                               "f_zero_1/add_2:0"], feed_dict={            
                                "p_phonemes:0": s['phonemes'], 
                                "p_seq_len:0": s['phonemes_seq_len'], 
                                "p_spk_ids:0": np.ones((batch_size))
                            })
            s['pred_voiced'] = pred_voiced
            s['pred_f_zero'] = pred_f_zero
        return sentences
    
    


In [8]:
d_test_result = duration_predict(duration_model_path,duration_test_sen)
f_test_result = frequency_predict(frequency_model_path,frequency_test_sen)

INFO:tensorflow:Restoring parameters from ../weights/train_duration_model_notebook/model.ckpt
INFO:tensorflow:Restoring parameters from ../weights/train_frequency_model_notebook/model.ckpt


In [6]:
"""

import sys
sys.path.append("..")
import os
import tensorflow as tf
import numpy as np
from preprocess import *

model_path = '../weights/train_grapheme_to_phoneme_model_notebook/model.ckpt'
def predict(model_path):
    tf.reset_default_graph()
    saver = tf.train.import_meta_graph(model_path+".meta")
    graph = tf.get_default_graph()

    with tf.Session() as sess:
        saver.restore(sess,model_path)
        pred = sess.run(prediction_tf, feed_dict={            
                                "pred_ch": np.random.rand(2, 200),
                                "pred_ch_n": 10* np.ones((2))
                            })
        return pred
print predict(model_path)
"""

'\n\nimport sys\nsys.path.append("..")\nimport os\nimport tensorflow as tf\nimport numpy as np\nfrom preprocess import *\n\nmodel_path = \'../weights/train_grapheme_to_phoneme_model_notebook/model.ckpt\'\ndef predict(model_path):\n    tf.reset_default_graph()\n    saver = tf.train.import_meta_graph(model_path+".meta")\n    graph = tf.get_default_graph()\n\n    with tf.Session() as sess:\n        saver.restore(sess,model_path)\n        pred = sess.run(prediction_tf, feed_dict={            \n                                "pred_ch": np.random.rand(2, 200),\n                                "pred_ch_n": 10* np.ones((2))\n                            })\n        return pred\nprint predict(model_path)\n'